In [2]:
%pip install stagehand      

  Using cached browserbase-1.4.0-py3-none-any.whl.metadata (15 kB)
  Using cached docstring_parser-0.17.0-py3-none-any.whl.metadata (3.5 kB)
  Using cached zipp-3.23.0-py3-none-any.whl.metadata (3.6 kB)
  Using cached pyee-13.0.0-py3-none-any.whl.metadata (2.9 kB)
   ---------------------------------------- 0.0/10.5 MB ? eta -:--:--
   -------------- ------------------------- 3.9/10.5 MB 19.6 MB/s eta 0:00:01
   ----------------------- ---------------- 6.3/10.5 MB 16.1 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.5 MB 14.8 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.5 MB 14.8 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.5 MB 14.8 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.5 MB 14.8 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.5 MB 14.8 MB/s eta 0:00:01
   ------------------------------ --------- 8.1/10.5 MB 14.8 MB/s eta 0:00:01
   ------------------------------- ------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import asyncio
import os
from typing import Optional
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from stagehand import StagehandConfig, Stagehand

# Load environment variables (ensure you have .env or set these in your shell)
load_dotenv()

# 1. Define the structure of the data we want to extract
class ImageInfo(BaseModel):
    image_url: str = Field(..., description="The direct URL to the source image file")
    alt_text: Optional[str] = Field(None, description="The descriptive text for the image")

async def get_image_link(query: str):
    """
    Uses Stagehand to find and extract the first image URL for a given query.
    """
    
    # Configure Stagehand
    # Note: Use 'LOCAL' if you are running locally without Browserbase
    config = StagehandConfig(
        env="LOCAL", 
        model_name="google/gemini-2.5-flash", # Or your preferred model
        model_api_key=os.getenv("GEMINI_API_KEY"),
    )

    stagehand = Stagehand(config)

    try:
        print(f"🔍 Searching for image: '{query}'...")
        await stagehand.init()
        page = stagehand.page

        # Navigate to Google Images directly with the query
        search_url = f"https://www.google.com/search?q={query}&tbm=isch"
        await page.goto(search_url)

        # Extract the image URL using the defined schema
        # Stagehand uses AI to find the correct element even if the DOM changes
        result = await page.extract(
            instruction="Extract the URL of the first high-quality image result in the grid.",
            schema=ImageInfo
        )

        return result

    except Exception as e:
        print(f"❌ An error occurred: {str(e)}")
        return None
    finally:
        await stagehand.close()

async def main():
    # Example usage:
    search_query = input("Enter what you want an image of: ") or "Cyberpunk city landscape"
    
    result = await get_image_link(search_query)
    
    if result and result.image_url:
        print("\n" + "="*50)
        print(f"✅ Found Image Link: {result.image_url}")
        if result.alt_text:
            print(f"📝 Description: {result.alt_text}")
        print("="*50)
    else:
        print("Could not find a valid image link.")

if __name__ == "__main__":
    # Ensure you have 'MODEL_API_KEY' set in your environment
    if not os.getenv("GEMINI_API_KEY"):
        print("⚠️ Warning: MODEL_API_KEY is not set in your environment variables.")
    
    await main()

🔍 Searching for image: 'image of apple'...
❌ An error occurred: 
Could not find a valid image link.
